### I hope you've enjoyed working with datasets in pandas so far!

Now that you've become familiar with how to do some basic data wrangling on a single dataset in **Python ** using **pandas**, we are going to move on to more complicated data operations.

In this lesson we will cover the following:

1. Working with text:
  * Splitting a column into multiple pieces
  * Extracting text
  * Converting between data types
* Converting categorical data into indicator variables
* Binning numerical data into categories
* Sorting datasets based on one or more columns
* Combining multiple datasets together coherently into a single dataset:
  * Joining/merging
* Gracefully finding and removing bad/incomplete data

We are going to go back to the Movielens 1M dataset, but we are going to fill in a significant amount of detail that was excluded initially by including data that was stored in separate files.

Specifically, we will be filling in/filtering/transforming information about the movies and users found in the `ratings.dat` dataset we worked on by combining that original info with data found in the `movies.dat` and `users.dat` files. These can be found in the same folder as where `ratings.dat` is located.

So, lets begin!

In [1]:
import pandas as pd
import numpy as np
from __future__ import print_function, unicode_literals, division

As always, you have to import the modules you will/may need.

Here are the schemas for the two new files we will work with:

1. `movies.dat`:
  * Row format: MovieID::Title::Genres
    * `MovieID`: The id of the movie, should correspond to values in `MovieID` column found in `ratings.dat`
    * `Title`: The title, in plain english, of the movie
    * `Genres`: A list of genres the movie is associated with, can be of varying length per row

2. `users.dat`:

  * Row format: UserID::Gender::Age::Occupation::Zip-code
    * `UserID`: The id of the user, should correspond to values found in `UserID` column foind in `ratings.dat`
    * `Gender`: Self-reported gender of the user, either `M` or `F`
    * `Age`: Self-reported age of the user
    * `Occupation`: Self-reported occupation category of the user
    * `Zip-code`: Self-reported zip-code of the user

Lets load these files in, along with the first file, `ratings.dat`, that we worked with:

In [2]:
movieData  = pd.read_csv("../data/movieData/movies.dat",sep = "::", names = ["MovieID","Title","Genres"])
userData   = pd.read_csv("../data/movieData/users.dat",sep = "::", names = ["UserID","Gender","Age","Occupation","Zip-code"])
ratingData = pd.read_csv("../data/movieData/ratings.dat",sep = "::", names = ['UserID','MovieID','Rating','Timestamp'])

/Users/markyashar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
/Users/markyashar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app
/Users/markyashar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by s

Let's take a look at the first few rows in the `movies.dat` dataset, which we've stored in a variable called `movieData` and then lets make sure that every movie is uniquely represented in this set:

In [3]:
movieData.head(20)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
print(movieData.shape)
print(movieData.MovieID.unique().shape)

So far, so good! There are as many rows in this dataset as there are unique movies!

**However**, we can clearly see that we will need to do some massaging with this dataset in order to get it into a more workable format: 
1. **Create a separate column for the year in which each movie was released by parsing the `Title` column (in case we want to do an analysis involving years)**
* **Parse the `Genres` column to extract every genre separately (this looks like it might be a bit tricky since there are different numbers of genres for each movie, but its actually really easy!)**

Lets try to tackle **1.** 

A reasonable approach here would be:

1. Creating a new column called `Year` by extracting the 4th-to-last through next-to-last characters from each Title 
* Converting this value from a `string` (or an `object`, as **pandas** calls them) into an `int`
* Removing the last 6 characters completely from the `Title` column

In [ ]:
print("Before formatting: ")
print(movieData.head())
movieData["Year"] = movieData.Title.str.slice(-5,-1) #1
movieData.Year = movieData.Year.astype(int) #2
movieData.Title = movieData.Title.str.slice(0,-7) #3
print("After formatting: ")
print(movieData.head())

Let's work through all 3 of the lines that didn't simply `print` to the screen.

There's a whole slew of functions that you can use to operate on `string` values that **pandas** provides, and the way to access them is very similar to the way you accessed all the cool time functionality when you called `dt` on a given row in the previous lesson.

So, to access `str` functions for a given column, just call `DataFrameName.columnName.str`, which translates in our case to `movieData.Title.str`.

The function that we are using here is called `slice`:

`movieData["Year"] = movieData.Title.str.slice(-5,-1)`

`slice` allows us to extract a "slice" of the string out directly, based on the starting and ending index (position) of the values we want "sliced" out in relation to the whole string.

**Indices for lists, arrays, strings, and collections of values more generally in Python always begin at 0 and end at the length of the string/list/collection-1. Indexing can also proceed from the end of the string/list/collection, etc. backwards, using negative numbers.**

So, to extract the 4th-to-last to next-to-last values in the string, we have to use index values of -5 and -1.

There are other ways to extract or split/transform strings in **Python** and **pandas** and I encourage you to look over the other functions found in the string module of **Python** and the kinds of `string` operations built into **pandas** itself. [Here's a useful link](http://pandas.pydata.org/pandas-docs/stable/text.html)

The second line shouldn't seem nearly as challenging:

`movieData.Year = movieData.Year.astype(int)`

We are just taking the column, and forcing the values in the column to behave as `int` types (numbers) and not as `string` types (on which we can't operate numerically).

### User-defined functions and 'apply'

Sometimes we want to apply a more complicated function to our data than what is already built into pandas or numpy. We can do this using the apply function.

Compare the following to the filter example we already discussed.

First I define an anonymous function that will capitalize a string.

In [ ]:
capitalizer = lambda row: row['Title'].upper()

In [ ]:
movieData['Capitalized Title'] = movieData.apply(capitalizer, axis = 1)
movieData.head()

With that out of the way, lets tackle the next data massaging task:

**Parse the `Genres` column to extract every genre separately (this looks like it might be a bit tricky since there are different numbers of genres for each movie, but its actually not as hard as one would think!)**

The approach we are going to take here is fairly common in Data Science pipelines and involves converting categorical variables (like our genres column) into a number-like representation. 

This is typically called creating **indicator variables** (you'll see why they're called indicator variables shortly).

The way this approach works is:

1. **Collect all of the distinct values found for the given categorical type (in our case, genre).**
* **Create a column per category value, and indicate whether the given value is present or absent for each row in the dataset.**

The caveat here is that if your categorical variable has many distinct values that it ranges over (>100), then the transformation to indicator variables can lead to a really sparse, large, and space-inefficient representation of your data (for 100 distinct values, your matrix would have to be at least 99 columns wide).

Thankfully, **pandas** can do both steps **1. and 2.** for you in one line of code, as long as the data is stored in the right format (which it is!).

Here is how this transformation from categorical column -> indicator variable would be executed in **pandas**:

In [ ]:
movieData.head()

In [ ]:
genresDF = movieData.Genres.str.get_dummies(sep = "|")
genresDF.head()

Awesome, so that totally converted the `Genres` column into the expanded indicator variable representation in one line of code!

Let's go through it:

`genresDF = movieData.Genres.str.get_dummies(sep = "|")`

We used `str.get_dummies()` on the `Genres` column and passed in a value to the `sep` parameter, telling the function that if a row had multiple entries, then the `|` character separated them. 

If, on the other hand, we only had a single genre per row, we could have left the parameter `sep` out completely and simply called:

`genresDF = movieData.Genres.str.get_dummies()`

### Joining datasets


But now we have a bit of a problem, all of these indicator variables are stored in a separate `DataFrame` from the original `movieData`. We can fix this by **joining the two datasets together.**

Join operations are very common operations in databases and in data processing pipelines more generally. **Join operations take two distinct datasets and combine them based on some common column or set of columns (called join keys) so that all columns from both datasets are combined into a single dataset based on the kind of join you want to perform.**

3 basic kinds of joins exist. To formalize the join types, lets say we have two datasets called A and B:

1. **One-sided (left or right) join**: This kind of join takes all the join keys from one of the datasets (lets say A), keeps all of them, and attempts to find all the rows in B with keys identical to only those in A. Any keys from A not found in B will have null values for columns in B.
* **Inner join**: This join only takes all the keys common to both A and B and combines all of the columns in both A and B together. Rows for which keys are not found in both A and B are thrown out.
* **Outer (full outer) join**: This join takes all the keys found in either A or B and combines both datasets on the keys that are common to both. For those keys that arent in both datasets, all columns remain, but have null values for the missing columns. (We will talk about null values soon).

I will not go into more detail about each of these kinds of joins now, but if youre interested in learning more about join types [this is a good explanation](http://blog.sweetlabs.com/2013/12/cheat-sheet-using-python-pandas-perform-fast-sql-like-joins/) and [here is the **pandas** documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html)

The exact type of join we will perform now actually doesn't matter, because all of them will lead to the same result (Do you know why?).

So, on to the join:

In [ ]:
movieDataWithGenres = movieData.merge(genresDF,left_index=True,right_index=True)
movieDataWithGenres.head()

To perform a join, you can call **pandas** `merge()` function and pass it a variety of parameters (read the documentation for more info). In our case:

`movieData = movieData.merge(genresDF,left_index=True,right_index=True)`

We are calling `merge` from `movieData`, so `movieData` functions as our left dataset (table), and `genresDF` functions as our right table. 

We are using the indices of both tables as join keys (thats what setting the parameters `left_index` and `right_index` to `True` does, as by default they are `False`).

If you were to join on something other than the index of both tables (like another column), you would have passed that column name (or names) as a `string` or a `List` of `string` names to the `left_on` and `right_on` parameters.

**However, in cases such as this, you don't have to do a join, but can simply concatenate the two dataframes together horizontally (since they are in exactly the same order):**

In [ ]:
movieDataWithGenres2 = pd.concat((movieData,genresDF),axis=1)
movieDataWithGenres2.head()

One last thing to do with this dataset! 
We still have the `Genres` column in `movieData`, but we've expanded it and don't need it.
So, let's delete it from the `DataFrame`.

In [ ]:
del movieDataWithGenres["Genres"]
movieDataWithGenres.head()

We've just learned about how to go from categorical data into indicator variables, which are effectively a kind of proto-numerical data. 

Now, lets learn how to transform data from a numerical value into a more-categorical value through binning. This process is also referred to as discretization (forcing some value into one of several distinct bins).

As always, we are going to learn how to do this by answering a specific question:

**In what decade were the most of the movies found in this dataset made?**

So, to attack this problem we would need to:

1. Extract all of the unique movies along with the year they were released
* Get the earliest year and latest year when a movie was made using `min` and `max` to get the range over which we are going to work
* Get the start of the decades that these years would have been a part of
* Create a new column that bins each year based on a decade-based binning from the earliest decade found to the last decade found from the previous step.
* Group by this new column and get the size of each bin.
* Sort the grouped data in descending order and look at the first entry

Here is the entire sequence of steps:

In [ ]:
yearsData = movieData[["Title","Year"]] #1
minYear = yearsData.Year.min() #2
maxYear = yearsData.Year.max() #3

print("Earliest year: %d Latest year: %d"%(minYear,maxYear)) #4

In [ ]:
minDecade = np.floor_divide(minYear,10)*10 #5
maxDecade = np.floor_divide(maxYear,10)*10 #6

print("Earliest decade: %d End of Final Decade: %d"%(minDecade,maxDecade)) #7

In [ ]:
allDecades = np.arange(minDecade,maxDecade+10,10) #8

print("All decades: %s" %(allDecades)) #9

In [ ]:
yearsData["Decade"] = pd.cut(yearsData.Year,allDecades) #10
yearsData
moviesPerDecade = yearsData.groupby("Decade").size() #11
moviesPerDecade.sort_values(ascending=False,inplace=True)   #12

print("Decade with the most movies:") #13
print(moviesPerDecade) #14

Lets work through all the steps, carefully going over those that you may appear new to you.

We are not going to go over **steps 1-3** as they should be familiar to you (extract the columns we are interested in, get min and max of one of those columns)

In **step 4**, we print the results of **2-3** out to the screen using some formatting tricks. 

The `%d` inside of the strings tells python that we are going to be putting some data (an integer) in there later (after the string has been written) and pass those values as a tuple after the string, prefixed with another `%` symbol. 

So, `%d` means "put an integer in there" and the `%(minYear,maxYear)` means "these are the two values, that you should treat as integers, you should put in, in the order they are written."

**Steps 5-6** are some trickery we have to use to get the closest decade before the minimum and maximum dates when the movies were released.

`np.floor_divide()` is a function that takes 2 arguments (a numerator and a denominator) and returns the first number divided by the second, with any remainder lobbed off. 

Our two arguments in both cases will be the years we found, and we are going to divide them by 10 and then multiply them by 10, so the remainder is lobbed off and the closest decade is returned!

**Step 7** is more printing magic that you should now understand a little bit.

**Step 8** creates the range of decades we are interested in. 

`np.arange` takes 3 arguments (the starting integer, the ending integer, and the spacing between them) and creates the evenly spaced range between the two numbers, but only inclusive of the first, and exclusive of the second. THIS IS WHY WE HAD TO ADD 10 TO THE `maxDecade` VALUE!

So:

    np.arange(0,10,2) 
    returns:
    [0,2,4,6,8]
    and in order to get: 
    [02,4,6,8,10] 
    we would need to pass:
    np.arange(0,12,2)

**Step 9** prints what **Step 8** did. Because `allDecades` is not an `int` but a sequence of `int` values (called an `ndarray`, but don't worry about this), we have to treat it as a `string`, hence the `%s` (which stands for `string`).

**Step 10** is the actual **pandas** code doing the work we are interested in. 

`cut` is a function that takes two parameters:

* The first parameter is a sequence of values, the `Year` column in our case
* The second parameter can either be a sequence of numbers or a single value, and does something different based on the kind of input you give. If the parameter is a sequence of values, it assigns each value in the first parameter to a bin that is exclusive on the left and inclusive on the right for the sequence you give (I will explain). If the second parameter is a single value, this tells **pandas** that the values in the first parameter should be placed into that number of bins, equally spaced around the range of values found in the first parameter (phew!)

Heres an example, lets say you have a list of numbers `dude = [1,20,30,43,100,26]`:

`pd.cut(dude,[0,10,100])`

will return:

`[(0,10],(10,100],(10,100],(10,100],(10,100],(10,100]]`

But then this:

`pd.cut(dude,3)`

will return:

`[(0.901, 34], (0.901, 34], (0.901, 34], (34, 67], (67, 100], (0.901, 34]]`

So in the first case, you give it the bins, in the second, **pandas** figures out the best way to split the data into the correct number of equally large bins.

Make sense?

So:

`yearsData["Decade"] = pd.cut(yearsData.Year,allDecades)`

creates a new column called `Decade` in the `yearsData` dataset and assigns to it the bins created by passing the `Year` column and the range of decades found in `allDecades`. 

If we had just wanted 9 equally spaced year ranges (instead of the 9 decades our dataset covers), we could have just done:

`yearsData["Decade"] = pd.cut(yearsData.Year,9)`

**Step 11** should be familiar to you by now. `groupby` on our new column, and get the size of every group and save it in a variable `moviesPerDecade`.

**Step 12** introduces sorting. 

`sort` is a pandas function that can sort your dataset along one or several columns in a variety of ways. 

The `ascending` parameter is by default `True` and forces the sort to either be from smallest value at the top to largest value at the bottom (`ascending = True`, the default) or largest first, smallest last (`ascending=False` as we have).

The `inplace` parameter makes it so that you can either sort the dataset and not have to reassign the sorted dataset to a new variable (`inplace = True`; this is especially useful when your dataset is very large and creating a sorted copy would create way too much memory pressure on your system) or not (`inplace = False`, the default).

So:

`moviesPerDecade.sort(ascending=False,inplace=True)`

Sorts this dataset as is (dont copy it somewhere else) and make sure its in descending order. Because this dataset is a `Series` object, which only contains a single column, we don't have to (and really can't) specify the column along which we want to sort.

**Steps 13 and 14** should also be obvious. **Step 13** prints a statement to the screen and **Step 14** simply prints the first row in the dataset (since its in descending order, its also the largest element!)

Make sense?

Ok, we're done working with `movieData` for now, lets move on to `userData`:

In [ ]:
userData.head()

Here is some explanation about the `Age` and `Occupation` fields (before you think there is someone that is claiming to be a 1 year old in the dataset!):

- Age is a categorical variable mapped to the following ranges:

    *  1:  "Under 18"
    * 18:  "18-24"
    * 25:  "25-34"
    * 35:  "35-44"
    * 45:  "45-49"
    * 50:  "50-55"
    * 56:  "56+"

- Occupation is a categorical variable mapped as follows:

    * 0:  "other" or not specified
    * 1:  "academic/educator"
    * 2:  "artist"
    * 3:  "clerical/admin"
    * 4:  "college/grad student"
    * 5:  "customer service"
    * 6:  "doctor/health care"
    * 7:  "executive/managerial"
    * 8:  "farmer"
    * 9:  "homemaker"
    * 10: "K-12 student"
    * 11: "lawyer"
    * 12: "programmer"
    * 13: "retired"
    * 14: "sales/marketing"
    * 15: "scientist"
    * 16: "self-employed"
    * 17: "technician/engineer"
    * 18: "tradesman/craftsman"
    * 19: "unemployed"
    * 20: "writer"


Ok, so then there are several combinations of `Age` and `Occupation` that we can remove:

1. **Make sure that every user mapped to `Age 1` has an `Occupation` `4 or 10`, if not, remove them.**
*  **Make sure no one that has an `Occupation` other than `10` is of `Age` either `1 or 18`, otherwise remove them.**
* **Make sure all of the zipcodes are in a standard (5 digit) format, but keep them as `string` types (any idea why?)**

Now that you're more familiar with **pandas**, you should figure out how to do all 3 of these and either remove or change the offending rows.


(**Hint:** Use `np.logical_and` to test for the occurence of multiple conditions for your mask that you are going to have to create! `np.logical_and` takes in 2 arguments, the two conditions you want satisfied.)

Store the new, filtered `DataFrame` in a separate variable called `cleanedUserData` and keep the cleaned zipcodes in the column `Zip-code`.

(**Hint:** I suggest you use the slicing logic we used before for the `Title` column to extract the first 5 values in the `Zip-code` column and then convert the column from a `string` to an `int`)

In [ ]:
#Answering question 1
#case where they are young
myMask1 = np.logical_and(userData.Age==1,np.logical_or(userData.Occupation==4,userData.Occupation==10))
#case where they are not young
myMask1part2 = np.logical_and(userData.Age!=1,np.logical_and(userData.Occupation!=4,userData.Occupation!=10))
myMask1Better = np.logical_or(myMask1,myMask1part2)

userDataMyFilteredTest = userData[myMask1Better]
print(userDataMyFilteredTest.head())
print(userData.head())
print(userDataMyFilteredTest)

In [ ]:
#Answering question2 using output of question 1
myMask2 = np.logical_or(np.logical_and(userDataMyFilteredTest.Occupation!=10,np.logical_and(userDataMyFilteredTest.Age!=1,userDataMyFilteredTest.Age!=18)),userDataMyFilteredTest.Occupation==10)
userDataFinallyFiltered = userDataMyFilteredTest[myMask2]
print(userDataFinallyFiltered.head())
print(userData.head())
#Answer question 3
userData["Zip-code"].unique()                     
userDataFinallyFiltered = userDataFinallyFiltered[userDataFinallyFiltered["Zip-code"].str.len()>=5]
userDataFinallyFiltered["Zip-code"] = userDataFinallyFiltered["Zip-code"].str.slice(0,5)

print("Before removing garbage:",userData.shape)
print("After removing garbage:",userDataFinallyFiltered.shape)

Ok, now that all of this data munging on both of these datasets is done, lets **join** them with the original `ratingsData`. 

To join each of these datasets, you will need to use a distinct set of **join keys:**

1. To join the `movieData` dataset to the `ratingData` dataset, you will need to join on the `MovieID` column.
2. To join the `cleanedUserData` dataset to the `ratingData` dataset, you will need to join on the `UserID` column.

Also, remember that we removed several bad rows from the `userData` to get the `cleanedUserData` so when we perform the default **join** (an *inner join*) with the `ratingData` we will get a smaller dataset out!

So, go ahead and perform the joins yourself and store the resulting, final joined dataset in a variable called `filledAndCleanedRatingData`

In [ ]:
filledAndCleanedUserAndRatingData = userDataFinallyFiltered.merge(ratingData,on="UserID")
print("Cleaned users joined with ratings:")
print(filledAndCleanedUserAndRatingData.head())
print("")
print("Cleaned users+ratings joined with movies:")
filledAndCleanedAllData = filledAndCleanedUserAndRatingData.merge(movieData,on = "MovieID")
print(filledAndCleanedAllData.head())

And now, just to get a bit more practice with munging and transforming data in pandas, answer the following questions:

1. What was the most frequently rated movie?
  * In january?
* What movie was most commonly rated in the mornings (before noon)? 
  * What was its average rating?
  * What about in the evenings? What was its average rating?
* Which user saw movies that belonged to the largest collection of distinct genres?
* Which movie did the most people disagree on (had the highest standard deviation in ratings)?
* Which movie had the worst average rating?
  * For only those movies that were rated at least 3 times?
* What was the average rating for Comedies?
* What was the average rating for movies per decade?
* Which decade other than the 90s had the highest average rating?
  * Which decade had the largest spread in ratings (including the 90s)?
* Create a new column on a per-movie basis called `RatingFrequency` following these guidelines:
  * Movies that were rated 3 times or fewer
  * Movies rated between 3 and 10 times
  * Movies rated more than 10 times (you can use `np.inf` as your last number in the range to get the range "10 or more")
* What was the average rating for comedies for each `RatingFrequency` type?

In [ ]:
pass

You should now be very comfortable with these features of **pandas**:

* Working with `string` input within columns
* Converting categorical data into indicator variables
* Converting numerical data into categorical data 
* Joining multiple tables together